In [2912]:
import numpy as np
import pandas as pd
from sklearn.impute import KNNImputer
import os

In [2913]:
pd.set_option('display.max_rows', 5000)
pd.set_option('display.max_columns', 5000)
pd.set_option('display.width', 5000)
pd.set_option('display.max_colwidth', 5000)

In [2914]:
# Move up 2 directories
data_directory = '../..' 

# Load the CSV files
asthma_df = pd.read_csv(os.path.join(data_directory, 'Data', 'astma.csv'))

# Drop a kid

That kid has more than 24 missing columns. The decision has been to delete him from the database.

In [2915]:
# delete rows with subjectNr
asthma_df = asthma_df.drop(asthma_df[asthma_df['SubjectNr'] == 2429672].index).reset_index().drop('index', axis=1)

Detect the columns with lots of Nan

In [2916]:
# Compute for all the columns, the percentages of Nan per column
# We will drop the columns that have more than 50% Nan
percentNan = []
for col in asthma_df.columns:
    nan_percentage = asthma_df[col].isnull().sum() / len(asthma_df[col]) * 100
    percentNan.append((nan_percentage, col))

percentNan.sort(reverse=True)

for el in percentNan:
    if el[0] > 50:
        print(el)

(98.33526906697638, 'steps03')
(98.29655439411536, 'steps04')
(97.67711962833914, 'steps02')
(96.36082075106465, 'steps05')
(96.36082075106465, 'steps01')
(92.1796360820751, 'EventDay')
(90.59233449477352, 'steps00')
(79.90708478513356, 'steps23')
(77.66163375919474, 'steps06')
(76.38404955478126, 'EOS_weight_tot')
(76.38404955478126, 'EOS_treatmentburden_tot')
(76.38404955478126, 'EOS_school_tot')
(76.38404955478126, 'EOS_respiratory_tot')
(76.38404955478126, 'EOS_physical_tot')
(76.38404955478126, 'EOS_health_tot')
(76.38404955478126, 'EOS_emotional_tot')
(76.38404955478126, 'EOS_digestion_tot')
(76.38404955478126, 'EOS_bodyimage_tot')
(75.30003871467285, 'Eosinophils')
(73.94502516453736, 'Symptomscore')
(69.68641114982579, 'Serum_IgE')
(69.26054974835463, 'ACD6')
(67.36353077816493, 'Pseudomonas')
(67.36353077816493, 'BASELINE_weight_tot')
(67.36353077816493, 'BASELINE_treatmentburden_tot')
(67.36353077816493, 'BASELINE_school_tot')
(67.36353077816493, 'BASELINE_respiratory_tot')
(

## Child exrtaction

Let's first separate the databese into chilferen. This will create on database per children in order to identify them more easily. This can be useful if one data is repeated only once per child. We could then spread the data through the child.£

In [2917]:
def extractChild(df):

    # Create a new database per children and save it into a list of all the same subject numbers.
    df_children = []    # List of all the children
    
    # Loop through all the subject numbers
    for subject in df['SubjectNr'].unique():
        df_children.append(df[df['SubjectNr'] == subject])
    
    return df_children

# Dealing with Missing Values

## Technique
About technique 2 Matthijs wrote this:

Is it possible to do your analyses with both options to see whether it impacts the performance of your models? If not, I would just use the ones marked ‘2’ anyway to include as much data as possible.


In [2918]:
#asthma_df['Technique'].value_counts()
#asthma_df = asthma_df.drop(asthma_df[asthma_df['Technique'] == 1].index).reset_index().drop('index', axis=1)

# IF WE DELETE THE WRONG TECHNIQUE HERE, ANY dadaset.loc[i] WILL GIVE AN ERROR AT THE FIRST MISSING ROW
# We need to reset the indicies to avoid this

## Issue with HR columns

In the asthma dataset there seems to be a problem with HR00 to HR23. Sometimes, the heart rate goes above a million sometimes. It seems like the first couple digits correspond with the expected heart rate. 

In [2919]:
# In the asthma dataset there seems to be a problem with HR00 to HR23
# The heart rate goes above a million sometimes
# It seems like the first couple digits correspond with the expected heart rate.
# 2 cases: HR below 100 and HR above 100

# Loop through each column in the dataframe
for col in asthma_df.loc[:, "HR00":"HR23"]:
    for i in range(len(asthma_df[col])):
        if (asthma_df.loc[i, col] > 1000):
            # Take first 2 digits of float
            first_2_digits = str(asthma_df.loc[i, col])[:2]
            temp_number = float(first_2_digits)
            # If heart rate lower than 20 it means it should be greater than 100 (assuming heart rates < 200)
            if temp_number < 20:
                val = str(asthma_df.loc[i, col])[:3] + '.' + str(asthma_df.loc[i, col])[3]
                asthma_df.loc[i, col] = float(val)
            else:
                val = str(asthma_df.loc[i, col])[:2] + '.' + str(asthma_df.loc[i, col])[2]
                asthma_df.loc[i, col] = float(val)
            pass

asthma_children = extractChild(asthma_df)

### Heart Rate

In [2920]:
# For everty sick child, go through the heart rate and replace the Nan values with KNN
# For the documentation of interpolation: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.interpolate.html

# # Loop through all the children
# for child in asthma_children:
#     # Replace the missing values in the Heart rate using KNN
#     child.loc[:, "HR00":"HR23"] = child.loc[:, "HR00":"HR23"].interpolate(method='nearest',order = 5, limit_direction='forward', axis=0, limit = 5)
#     # pass

# child = asthma_children[0].interpolate(method='linear', limit_direction='forward', axis=1, limit = 3)


    

In [2921]:
for child in asthma_children:
    imputer = KNNImputer()          # n_neighbors=2
    child.loc[:, "HR00":"HR23"] = imputer.fit_transform(child.loc[:, "HR00":"HR23"])


/var/folders/cj/lqwb00w969q7gkbs7700mm300000gn/T/ipykernel_69173/2696809025.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  child.loc[:, "HR00":"HR23"] = imputer.fit_transform(child.loc[:, "HR00":"HR23"])
/var/folders/cj/lqwb00w969q7gkbs7700mm300000gn/T/ipykernel_69173/2696809025.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  child.loc[:, "HR00":"HR23"] = imputer.fit_transform(child.loc[:, "HR00":"HR23"])
/var/folders/cj/lqwb00w969q7gkbs7700mm300000gn/T/ipykernel_69173/2696809025.py:3: SettingWithC

In [2922]:
# print the HR columns
asthma_children[10].loc[:, "HR00":"HR23"]

asthma_df = pd.concat(asthma_children)

## AVGHR_daily

In [2923]:
# print the indexes of the nans in AVGHR_daily
index = asthma_df[asthma_df['AVGHR_daily'].isnull()].index

print(index)

for ind in index:
    # compute the average number of steps
    asthma_df.loc[ind, 'AVGHR_daily'] = (asthma_df.loc[ind, "HR00":"HR23"].mean())

asthma_df[asthma_df['AVGHR_daily'].isnull()].index

Int64Index([  86,  344,  346,  347,  404,  440,  441,  442,  443,  444,
            ...
            2302, 2376, 2444, 2445, 2446, 2447, 2448, 2449, 2466, 2484], dtype='int64', length=141)


Int64Index([], dtype='int64')

## HR05Perc & HR95Perc

In [2924]:
index05 = asthma_df[asthma_df['HR05Perc'].isnull()].index
index95 = asthma_df[asthma_df['HR95Perc'].isnull()].index

print(index05)

for ind in index05:
    # compute the average number of steps
    asthma_df.loc[ind, 'HR05Perc'] = (asthma_df.loc[ind, "HR00":"HR23"].quantile(0.05))

for ind in index95:
    # compute the average number of steps
    asthma_df.loc[ind, 'HR95Perc'] = (asthma_df.loc[ind, "HR00":"HR23"].quantile(0.95))

Int64Index([  86,  344,  346,  347,  404,  440,  441,  442,  443,  444,
            ...
            2302, 2376, 2444, 2445, 2446, 2447, 2448, 2449, 2466, 2484], dtype='int64', length=141)


## Steps

In [2925]:
# asthma_children[10].loc[:, "SubjectNr"]

asthma_children = extractChild(asthma_df)
asthma_children[10].loc[:, "steps00":"steps23"]

,steps00,steps01,steps02,steps03,steps04,steps05,steps06,steps07,steps08,steps09,steps10,steps11,steps12,steps13,steps14,steps15,steps16,steps17,steps18,steps19,steps20,steps21,steps22,steps23
290,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,13.0,NaN,NaN,1017.0,1474.0,1419.0,397.0,472.0,132.0,449.0,NaN,NaN,NaN
291,NaN,NaN,NaN,NaN,NaN,88.0,605.0,687.0,1265.0,976.0,758.0,107.0,1157.0,615.0,1264.0,494.0,42.0,283.0,323.0,25.0,128.0,NaN,NaN,NaN
292,NaN,NaN,NaN,NaN,NaN,NaN,63.0,101.0,265.0,1175.0,1057.0,1094.0,369.0,248.0,138.0,322.0,35.0,183.0,211.0,301.0,279.0,79.0,50.0,7.0
293,NaN,NaN,NaN,NaN,NaN,72.0,92.0,766.0,451.0,190.0,274.0,309.0,224.0,125.0,279.0,47.0,128.0,173.0,238.0,352.0,432.0,NaN,NaN,NaN
294,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,296.0,449.0,258.0,698.0,82.0,176.0,374.0,1313.0,825.0,1037.0,80.0,158.0,234.0,103.0,131.0,NaN
295,NaN,NaN,NaN,NaN,NaN,NaN,130.0,462.0,406.0,70.0,973.0,166.0,1593.0,359.0,352.0,644.0,317.0,601.0,1066.0,336.0,191.0,NaN,NaN,NaN
296,NaN,NaN,NaN,NaN,NaN,NaN,97.0,408.0,36.0,108.0,197.0,18.0,94.0,59.0,182.0,114.0,172.0,342.0,730.0,1217.0,492.0,243.0,207.0,42.0
297,NaN,NaN,NaN,NaN,NaN,20.0,57.0,194.0,409.0,162.0,187.0,67.0,60.0,678.0,494.0,1135.0,102.0,326.0,362.0,425.0,27.0,23.0,NaN,NaN
298,NaN,NaN,NaN,NaN,NaN,NaN,27.0,124.0,102.0,11.0,120.0,NaN,327.0,188.0,NaN,NaN,76.0,329.0,765.0,70.0,579.0,6.0,NaN,NaN
299,NaN,NaN,NaN,NaN,NaN,NaN,NaN,346.0,355.0,12.0,224.0,81.0,92.0,96.0,112.0,952.0,583.0,1366.0,563.0,310.0,45.0,6.0,NaN,NaN


In [2926]:
for child in asthma_children:
    child.loc[:, "steps00":"steps10"] = child.loc[:, "steps00":"steps10"].fillna(0.0)
    child.loc[:, "steps22":"steps23"] = child.loc[:, "steps22":"steps23"].fillna(0.0)


/var/folders/cj/lqwb00w969q7gkbs7700mm300000gn/T/ipykernel_69173/211351183.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  child.loc[:, "steps00":"steps10"] = child.loc[:, "steps00":"steps10"].fillna(0.0)
/var/folders/cj/lqwb00w969q7gkbs7700mm300000gn/T/ipykernel_69173/211351183.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  child.loc[:, "steps22":"steps23"] = child.loc[:, "steps22":"steps23"].fillna(0.0)


In [2927]:
asthma_children[59].loc[:, "steps00":"steps23"]

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/IPython/core/displayhook.py:275: UserWarning: Output cache limit (currently 1000 entries) hit.
Flushing oldest 200 entries.
  warn('Output cache limit (currently {sz} entries) hit.\n'


,steps00,steps01,steps02,steps03,steps04,steps05,steps06,steps07,steps08,steps09,steps10,steps11,steps12,steps13,steps14,steps15,steps16,steps17,steps18,steps19,steps20,steps21,steps22,steps23
1711,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,30.0,1376.0,511.0,570.0,284.0,270.0,1834.0,1708.0,349.0,276.0,0.0,0.0
1712,0.0,0.0,0.0,0.0,0.0,0.0,29.0,137.0,454.0,714.0,1963.0,419.0,207.0,469.0,76.0,102.0,944.0,330.0,70.0,418.0,218.0,20.0,0.0,0.0
1713,0.0,0.0,0.0,0.0,0.0,0.0,0.0,255.0,60.0,504.0,493.0,164.0,108.0,911.0,2705.0,2613.0,1065.0,467.0,60.0,327.0,277.0,9.0,0.0,0.0
1714,0.0,0.0,0.0,0.0,0.0,0.0,0.0,99.0,345.0,283.0,1425.0,125.0,783.0,318.0,275.0,827.0,105.0,40.0,178.0,393.0,198.0,NaN,0.0,0.0
1715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,190.0,483.0,132.0,315.0,176.0,243.0,939.0,1381.0,942.0,251.0,1838.0,4310.0,2465.0,112.0,117.0,9.0,0.0
1716,0.0,0.0,0.0,0.0,0.0,0.0,0.0,116.0,723.0,996.0,369.0,315.0,766.0,184.0,1009.0,100.0,1864.0,553.0,197.0,299.0,258.0,NaN,0.0,0.0
1717,0.0,0.0,0.0,0.0,0.0,26.0,31.0,144.0,1339.0,533.0,211.0,2159.0,943.0,459.0,328.0,1475.0,217.0,1059.0,415.0,662.0,307.0,NaN,13.0,0.0
1718,0.0,0.0,0.0,0.0,0.0,0.0,0.0,137.0,481.0,179.0,474.0,277.0,1016.0,670.0,954.0,257.0,529.0,384.0,264.0,648.0,21.0,307.0,0.0,0.0
1719,0.0,0.0,0.0,0.0,0.0,0.0,0.0,74.0,95.0,446.0,441.0,382.0,211.0,126.0,133.0,402.0,124.0,132.0,179.0,39.0,185.0,134.0,0.0,0.0
1720,0.0,0.0,0.0,0.0,0.0,0.0,0.0,107.0,79.0,973.0,252.0,411.0,171.0,92.0,951.0,529.0,106.0,217.0,49.0,272.0,319.0,471.0,0.0,0.0


In [2928]:
for child in asthma_children:
    try:
        imputer = KNNImputer()          # n_neighbors=2
        child.loc[:, "steps00":"steps23"] = imputer.fit_transform(child.loc[:, "steps00":"steps23"])
# asthma_children[59][10:].loc[:, "steps00":"steps23"] = imputer.fit_transform(asthma_children[59][10:].loc[:, "steps00":"steps23"])
    except:
        pass

# combine the child dataframes into one dataframe
asthma_df = pd.concat(asthma_children)
asthma_df.loc[:, "steps00":"steps23"] = imputer.fit_transform(asthma_df.loc[:, "steps00":"steps23"])

/var/folders/cj/lqwb00w969q7gkbs7700mm300000gn/T/ipykernel_69173/1996037095.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  child.loc[:, "steps00":"steps23"] = imputer.fit_transform(child.loc[:, "steps00":"steps23"])
/var/folders/cj/lqwb00w969q7gkbs7700mm300000gn/T/ipykernel_69173/1996037095.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  child.loc[:, "steps00":"steps23"] = imputer.fit_transform(child.loc[:, "steps00":"steps23"])
/var/folders/cj/lqwb00w969q7gkbs7700mm300000gn/T/ipykernel_69173/19960

TODO: Do the KNN with steps and heart rate at the same time.

## Step total daily & stepsTotalDetailed

In [2929]:
# print the indexes of the nans in stepsTotalDaily
index = asthma_df[asthma_df['stepsTotalDaily'].isnull()].index

for ind in index:
    # compute the total number of steps
    asthma_df.loc[ind, 'stepsTotalDaily'] = (asthma_df.loc[ind, "steps00":"steps23"].sum())

In [2930]:
# print the indexes of the nans in stepsTotalDetailed
index = asthma_df[asthma_df['stepsTotalDetailed'].isnull()].index

for ind in index:
    # compute the total number of steps
    asthma_df.loc[ind, 'stepsTotalDetailed'] = asthma_df.loc[ind]["stepsTotalDaily"]

## steps_hour_max

In [2931]:
# print the indexes of the nans in stepsTotalDetailed
index = asthma_df[asthma_df['steps_hour_max'].isnull()].index

for ind in index:
    # compute the max number of steps
    asthma_df.loc[ind, 'steps_hour_max'] = asthma_df.loc[ind]["steps00":"steps23"].max()

## steps15_19

In [2932]:
# print the indexes of the nans in steps15_19
index = asthma_df[asthma_df['steps15_19'].isnull()].index

for ind in index:
    # compute the max number of steps
    asthma_df.loc[ind, 'steps15_19'] = asthma_df.loc[ind]["steps15":"steps18"].sum()

## Weight and height

I assume that the child is a man. The assumption could have been done because the two sex on average have the same height, but only 1.5 kg of difference.

In [2933]:
asthma_df.loc[np.isnan(asthma_df['Weight']), ['Age','Weight','Height','Gender']]

index = asthma_df.loc[np.isnan(asthma_df['Weight']), ['Weight']].index

for ind in index:
    asthma_df.loc[ind, 'Weight'] = 39.92
    asthma_df.loc[ind, 'Height'] = 149.1

## PM25

The average value has been proven to be the best. (see link below)

In [2934]:
asthma_df[['PM25']].isna().sum()

# print the indexes where PM25 is Nan
asthma_df.loc[np.isnan(asthma_df['PM25']), ['PM25']]

# print Pm25 column
asthma_df['PM25']

0        13.04
1         3.09
2         4.94
3        12.79
4         3.31
5         2.83
6         5.36
7         8.23
8        10.17
9        16.34
10       22.08
11       20.55
12       18.76
13        3.85
14       11.30
15       20.79
16       32.33
17       24.28
18       25.54
19       25.07
20       26.89
21       27.63
22       11.76
23        3.96
24        4.60
25        5.59
26        1.93
27        6.06
28        1.50
29       20.79
30       32.33
31       24.28
32       25.54
33       25.07
34       26.89
35       27.63
36       11.76
37        3.96
38        4.60
39        5.59
40        1.93
41        6.06
42        1.50
43       11.45
44        3.47
45        0.70
46        6.11
47        4.40
48        2.84
49        3.42
50        8.81
51       10.94
52       16.37
53       19.59
54       17.61
55        7.71
56       10.98
57        5.29
58        8.23
59       11.30
60       20.79
61       32.33
62       24.28
63       25.54
64       25.07
65       26.89
66       2

In [2935]:
asthma_children = extractChild(asthma_df)

In [2936]:
# https://www.researchgate.net/publication/237537115_Estimation_of_missing_values_in_air_pollution_data_using_single_imputation_techniques
# for child in asthma_children:
#     # get index of Nan in child
#     index = child.loc[np.isnan(child['PM25']), ['PM25']].index
#     if index != 0:
#     # Replace the missing values in the PM25 with the mean of its neighbours
#         child.loc[index, 'PM25'] = (child.loc[index-1, 'PM25'].values + child.loc[index+1, 'PM25'].values) / 2

index = asthma_df.loc[np.isnan(asthma_df['PM25']), ['PM25']].index

for ind in index:
    asthma_df.loc[ind, 'PM25'] = (asthma_df.loc[ind-1, 'PM25'] + asthma_df.loc[ind+1, 'PM25']) / 2


In [2937]:
print(asthma_df['PM25'].isnull().sum())

0


## PM10

We do the same as above

In [2938]:
# Print the number of Missing values in the PM10 column
print(asthma_df[['PM10']].isna().sum())

PM10    11
dtype: int64


In [2939]:
index = asthma_df.loc[np.isnan(asthma_df['PM10']), ['PM10']].index

for ind in index:
    asthma_df.loc[ind, 'PM10'] = (asthma_df.loc[ind-1, 'PM10'] + asthma_df.loc[ind+1, 'PM10']) / 2

In [2940]:
print(asthma_df[['PM10']].isna().sum())

PM10    0
dtype: int64


## AgeDiagnosis

This one cannot be replaced because we have no information about that kid

In [2941]:
print(asthma_df[['AgeDiagnosis']].isna().sum())

# print the index of the missing values in the AgeDiagnosis column
index = asthma_df.loc[np.isnan(asthma_df['AgeDiagnosis']), ['AgeDiagnosis']].index
print(asthma_df.loc[index, ['SubjectNr','Age', 'AgeDiagnosis']])



AgeDiagnosis    29
dtype: int64
      SubjectNr  Age  AgeDiagnosis
1044    8364553    9           NaN
1045    8364553    9           NaN
1046    8364553    9           NaN
1047    8364553    9           NaN
1048    8364553    9           NaN
1049    8364553    9           NaN
1050    8364553    9           NaN
1051    8364553    9           NaN
1052    8364553    9           NaN
1053    8364553    9           NaN
1054    8364553    9           NaN
1055    8364553    9           NaN
1056    8364553    9           NaN
1057    8364553    9           NaN
1058    8364553    9           NaN
1059    8364553    9           NaN
1060    8364553    9           NaN
1061    8364553    9           NaN
1062    8364553    9           NaN
1063    8364553    9           NaN
1064    8364553    9           NaN
1065    8364553    9           NaN
1066    8364553    9           NaN
1067    8364553    9           NaN
1068    8364553    9           NaN
1069    8364553    9           NaN
1070    8364553    9   

## CONDITION

Same as above all the nan refer to one kid. Thus, we assumed it was controlled

In [2942]:
print(asthma_df[['CONDITION']].isna().sum())

# print the index of the missing values in the AgeDiagnosis column using pd.isnull()
index = asthma_df.loc[pd.isnull(asthma_df['CONDITION']), ['CONDITION']].index
asthma_df.loc[index, ['CONDITION']] = 'Controlled asthma'
print(asthma_df.loc[index, ['SubjectNr', 'CONDITION']])



CONDITION    0
dtype: int64
Empty DataFrame
Columns: [SubjectNr, CONDITION]
Index: []


## DayCare

Replace by 0

In [2943]:
print(asthma_df[['DayCare']].isna().sum())
print(asthma_df[asthma_df['DayCare'] != 0].shape[0])

# print the index of the missing values in the AgeDiagnosis column using pd.isnull()
index = asthma_df.loc[pd.isnull(asthma_df['DayCare']), ['DayCare']].index
asthma_df.loc[index, ['DayCare']] = 0
print(asthma_df.loc[index, ['SubjectNr', 'DayCare']])

DayCare    0
dtype: int64
0
Empty DataFrame
Columns: [SubjectNr, DayCare]
Index: []


## Dis_Chronic

In [2944]:
# Count the number of rows that have a different value than 0
print(asthma_df[['Dis_Chronic']].isna().sum())
print(asthma_df[asthma_df['Dis_Chronic'] != 0].shape[0])

# print the index of the missing values in the AgeDiagnosis column using pd.isnull()
index = asthma_df.loc[pd.isnull(asthma_df['Dis_Chronic']), ['Dis_Chronic']].index
asthma_df.loc[index, ['Dis_Chronic']] = 0
# print(asthma_df.loc[index, ['SubjectNr', 'Dis_Chronic']])


Dis_Chronic    0
dtype: int64
0


## Hosp_tratme

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!  DID NOT FINISH IT !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
Don't really know how to handle it.

In [2945]:
# Count the number of rows that have a different value than 0
# print(asthma_df[['Hosp_treatme']].isna().sum())

for i in range(len(asthma_df)):
    if asthma_df.loc[i, 'Hosp_treatme'] != 0:
        print(i, asthma_df.loc[i, ['Hosp_treatme', 'SubjectNr']])

# print(asthma_df[asthma_df['Hosp_treatme'] != 0].shape[0])

# print the index of the missing values in the AgeDiagnosis column using pd.isnull()
# index = asthma_df.loc[pd.isnull(asthma_df['Hosp_treatme']), ['Hosp_treatme']].index
# asthma_df.loc[index, ['Hosp_treatme']] = 0


1886 Hosp_treatme        1.0
SubjectNr       4745136
Name: 1886, dtype: object
1887 Hosp_treatme        1.0
SubjectNr       4745136
Name: 1887, dtype: object
1888 Hosp_treatme        1.0
SubjectNr       4745136
Name: 1888, dtype: object
1889 Hosp_treatme        1.0
SubjectNr       4745136
Name: 1889, dtype: object
1890 Hosp_treatme        1.0
SubjectNr       4745136
Name: 1890, dtype: object
1891 Hosp_treatme        1.0
SubjectNr       4745136
Name: 1891, dtype: object
1892 Hosp_treatme        1.0
SubjectNr       4745136
Name: 1892, dtype: object
1893 Hosp_treatme        1.0
SubjectNr       4745136
Name: 1893, dtype: object
1894 Hosp_treatme        1.0
SubjectNr       4745136
Name: 1894, dtype: object
1895 Hosp_treatme        1.0
SubjectNr       4745136
Name: 1895, dtype: object
1896 Hosp_treatme        1.0
SubjectNr       4745136
Name: 1896, dtype: object
1897 Hosp_treatme        1.0
SubjectNr       4745136
Name: 1897, dtype: object
1898 Hosp_treatme        1.0
SubjectNr       4745136

 Hosp_treatme        1.0
SubjectNr       7367843
Name: 2313, dtype: object
2314 Hosp_treatme        1.0
SubjectNr       7367843
Name: 2314, dtype: object
2315 Hosp_treatme        1.0
SubjectNr       7367843
Name: 2315, dtype: object
2316 Hosp_treatme        1.0
SubjectNr       7367843
Name: 2316, dtype: object
2317 Hosp_treatme        1.0
SubjectNr       7367843
Name: 2317, dtype: object
2318 Hosp_treatme        1.0
SubjectNr       7367843
Name: 2318, dtype: object
2319 Hosp_treatme        1.0
SubjectNr       7367843
Name: 2319, dtype: object
2320 Hosp_treatme        1.0
SubjectNr       7367843
Name: 2320, dtype: object
2321 Hosp_treatme        1.0
SubjectNr       7367843
Name: 2321, dtype: object
2322 Hosp_treatme        1.0
SubjectNr       2901469
Name: 2322, dtype: object
2323 Hosp_treatme        1.0
SubjectNr       2901469
Name: 2323, dtype: object
2324 Hosp_treatme        1.0
SubjectNr       2901469
Name: 2324, dtype: object
2325 Hosp_treatme        1.0
SubjectNr       2901469
Nam

## Gender

In [2946]:
# print the index of the missing values in the AgeDiagnosis column using pd.isnull()
index = asthma_df.loc[pd.isnull(asthma_df['Gender']), ['Gender']].index
print(asthma_df.loc[index, ['SubjectNr', 'Gender']])

Empty DataFrame
Columns: [SubjectNr, Gender]
Index: []


## Hospital_Trt

In [2947]:
# print the index of the missing values in the AgeDiagnosis column using pd.isnull()
index = asthma_df.loc[pd.isnull(asthma_df['Hospital_Trt']), ['Hospital_Trt']].index
print(asthma_df.loc[index, ['SubjectNr', 'Hospital_Trt']])

Empty DataFrame
Columns: [SubjectNr, Hospital_Trt]
Index: []


## PetsHome

In [2948]:
# print the index of the missing values in the AgeDiagnosis column using pd.isnull()
index = asthma_df.loc[pd.isnull(asthma_df['PetsHome']), ['PetsHome']].index
print(asthma_df.loc[index, ['SubjectNr', 'PetsHome']])

Empty DataFrame
Columns: [SubjectNr, PetsHome]
Index: []


## Family_his

Still need to be done !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

In [2949]:
asthma_children = extractChild(asthma_df)
countYes = 0
countNo = 0
for child in asthma_children:
    # verify the value of the first row of the child
    if child.iloc[1]['Family_his'] == "Yes":
        countYes += 1
    elif child.iloc[1]['Family_his'] == "No":
        countNo += 1

print(countYes)
print(countNo)

48
40


## SchoolYear & school_year_edit

The missing values refere to one kid. Thus, I put it at the same grade as a 7 year old kid should be (I assume that he did not fail a class).


In [2950]:
# Primary school year 4

index1 = asthma_df.loc[pd.isnull(asthma_df['SchoolYear']), ['SchoolYear']].index
asthma_df.loc[index1, ['SchoolYear']] = 4
# print(asthma_df.loc[index1, ['SubjectNr', 'SchoolYear']])

index2 = asthma_df.loc[pd.isnull(asthma_df['school_year_edit']), ['school_year_edit']].index
asthma_df.loc[index2, ['school_year_edit']] = 'Primary school year 4'
# print(asthma_df.loc[index2, ['SubjectNr', 'school_year_edit']])

## Sports

In [2951]:
index1 = asthma_df.loc[pd.isnull(asthma_df['Sports']), ['Sports']].index
print(asthma_df.loc[index1, ['SubjectNr', 'Sports']])

      SubjectNr Sports
1682    5578891    NaN
1683    5578891    NaN
1684    5578891    NaN
1685    5578891    NaN
1686    5578891    NaN
1687    5578891    NaN
1688    5578891    NaN
1689    5578891    NaN
1690    5578891    NaN
1691    5578891    NaN
1692    5578891    NaN
1693    5578891    NaN
1694    5578891    NaN
1695    5578891    NaN
1696    5578891    NaN
1697    5578891    NaN
1698    5578891    NaN
1699    5578891    NaN
1700    5578891    NaN
1701    5578891    NaN
1702    5578891    NaN
1703    5578891    NaN
1704    5578891    NaN
1705    5578891    NaN
1706    5578891    NaN
1707    5578891    NaN
1708    5578891    NaN
1709    5578891    NaN
1710    5578891    NaN
1827    5039256    NaN
1828    5039256    NaN
1829    5039256    NaN
1830    5039256    NaN
1831    5039256    NaN
1832    5039256    NaN
1833    5039256    NaN
1834    5039256    NaN
1835    5039256    NaN
1836    5039256    NaN
1837    5039256    NaN
1838    5039256    NaN
1839    5039256    NaN
1840    503

## Wear information

When did the kid wear the watch. Such info is not useful to predict the asthma crisis.

In [2952]:
asthma_df = asthma_df.drop(['wear24H', 'wear23', 'wear22', 'wear21', 'wear20', 'wear19','wear18','wear17','wear16H','wear16','wear15',
                                       'wear14','wear13','wear12','wear11','wear10','wear09','wear08','wear07','wear06','wear05','wear04','wear03',
                                       'wear02','wear01','wear00'], axis=1)

## waketimeReport - bedtimeReport - totalSleepDuration

It brings the missing data from 50% to 0-3%

In [2953]:
for index, row in asthma_df.iterrows():
    if row['DayNo'] == 0: # we guess it or we drop the row (but we have to guess the steps as well then)
        if not pd.isna(row['waketimeReport']):
            asthma_df.loc[index, 'waketimeReport'] = np.nan
        continue
    waketime = 0
    for stepsXX in asthma_df.loc[:, 'steps00':'steps11']:
        if row[stepsXX] > 0:
            if not np.isnan(asthma_df.loc[index-1, 'totalSleepDuration']): # i.e. the kid went to sleep before midnight
                asthma_df.loc[index, 'waketimeReport'] = waketime # e.g. 8 means the kid woke up between 8:00-9:00 -> steps08
                sleeptime = int(23 - (np.round(asthma_df.loc[index-1, 'totalSleepDuration']/60/60) - waketime)) # when a kid go to sleep after midnight we get >24 numbers, but I think it's okay
                asthma_df.loc[index-1, 'bedtimeReport'] = sleeptime # e.g. 22 means the kid went to sleep between 22:00-23:00 -> steps22
            
            elif stepsXX != 'steps00': # i.e. the kid went to sleep before midnight but we have no totalSleepDuration data about the night
                asthma_df.loc[index, 'waketimeReport'] = waketime
                sleeptime = 23
                for stepsXX_sleep in reversed(asthma_df.loc[:, 'steps00':'steps23'].columns):
                    if asthma_df.loc[index-1, stepsXX_sleep] > 0:
                        asthma_df.loc[index-1, 'bedtimeReport'] = sleeptime
                        asthma_df.loc[index-1, 'totalSleepDuration'] = (23 - sleeptime + waketime) * 60 * 60
                        break
                    sleeptime -= 1
            
            else:
                for stepsXX_sleep in asthma_df.loc[:, stepsXX:'steps11']: # i.e. the kid went to sleep after midnight
                    if row[stepsXX_sleep] == 0:
                        sleeptime = waketime + 24 - 1 # if we change it to 0..5 instead of 24..29, we still need to subtrack 1
                        asthma_df.loc[index-1, 'bedtimeReport'] = sleeptime
                        for stepsXX_wake in asthma_df.loc[:, stepsXX_sleep:'steps11']:
                            if row[stepsXX_wake] > 0:
                                asthma_df.loc[index, 'waketimeReport'] = waketime
                                asthma_df.loc[index-1, 'totalSleepDuration'] = (sleeptime - 24 + waketime) * 60 * 60
                                # Since we start the for loop with stepsXX again, waketime will be +1 higher.
                                # It means that the number of sleeping hours won't be equal to the number of 0 steps,
                                # but 1 more. This is actually not a problem, since the method that tries to guess the
                                # waketime/sleeptimeReport also not perfect due to the 'round' function.
                                # Leave it like this, it's maybe more realistic.
                                break
                            waketime += 1
                        break
                    waketime += 1
            break
        waketime += 1
    
    if row['DayNo'] == 28 or (row['DayNo'] == 27 and ...):
        hour = 23        
        for stepsXX_sleep in reversed(asthma_df.loc[:, 'steps00':'steps23'].columns):
            if asthma_df.loc[index, stepsXX_sleep] > 0:
                asthma_df.loc[index, 'bedtimeReport'] = hour
                break
            hour -= 1

In [2954]:
asthma_children = extractChild(asthma_df)

for child in asthma_children:
    imputer = KNNImputer()
    child.loc[:,["waketimeReport"]] = np.round(imputer.fit_transform(child.loc[:,["waketimeReport"]]))

asthma_df = pd.concat(asthma_children)

/var/folders/cj/lqwb00w969q7gkbs7700mm300000gn/T/ipykernel_69173/1222426837.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  child.loc[:,["waketimeReport"]] = np.round(imputer.fit_transform(child.loc[:,["waketimeReport"]]))
/var/folders/cj/lqwb00w969q7gkbs7700mm300000gn/T/ipykernel_69173/1222426837.py:5: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  child.loc[:,["waketimeReport"]] = np.round(imputer.fit_transform(child.loc[:,["waketimeReport"]]))
/var/folders/cj/lqwb00w969q7gkbs7700mm300000gn/T/ipykernel_69173/1222426837.py:5

## HRAvgSleep (avg hr during sleeping), HRMinSleep, HRMaxSleep
The original ones look good, we should keep them

In [2955]:
cols = asthma_df.loc[:, 'HR00':'HR23'].columns

for index, row in asthma_df.iterrows():
    if np.isnan(row['HRAvgSleep']):
        if row['DayNo'] == 28:
            continue
    
        bedtimeReport = asthma_df.loc[index,'bedtimeReport']
        waketimeReport = int(asthma_df.loc[index+1,'waketimeReport'])

        if np.isnan(waketimeReport):
            continue

        if bedtimeReport > 22:
            bedtimeReport -= 24

        night1 = asthma_df.loc[index, cols[bedtimeReport+1]:'HR23']
        night2 = asthma_df.loc[index, 'HR00':cols[waketimeReport-1]]

        night1_sum = night1.sum()
        night1_len = len(cols[bedtimeReport+1:])
        night2_sum = night2.sum()
        night2_len = len(cols[:waketimeReport])

        asthma_df.loc[index, 'HRAvgSleep'] = (night1_sum+night2_sum)/(night1_len+night2_len)

    if np.isnan(row['HRMinSleep']):
        if row['DayNo'] == 28:
            continue

        bedtimeReport = asthma_df.loc[index,'bedtimeReport']
        waketimeReport = int(asthma_df.loc[index+1,'waketimeReport'])

        if np.isnan(waketimeReport):
            continue

        if bedtimeReport > 22:
            bedtimeReport -= 24

        night1 = asthma_df.loc[index, cols[bedtimeReport+1]:'HR23']
        night2 = asthma_df.loc[index, 'HR00':cols[waketimeReport-1]]

        night1_min = night1.min()
        night2_min = night2.min()
        
        asthma_df.loc[index, 'HRMinSleep'] = np.minimum(night1_min, night2_min)

    if np.isnan(row['HRMaxSleep']):
        if row['DayNo'] == 28:
            continue
    
        bedtimeReport = asthma_df.loc[index,'bedtimeReport']
        waketimeReport = int(asthma_df.loc[index+1,'waketimeReport'])

        if np.isnan(waketimeReport):
            continue

        if bedtimeReport > 22:
            bedtimeReport -= 24

        night1 = asthma_df.loc[index, cols[bedtimeReport+1]:'HR23']
        night2 = asthma_df.loc[index, 'HR00':cols[waketimeReport-1]]

        night1_max = night1.max()
        night2_max = night2.max()

        asthma_df.loc[index, 'HRMaxSleep'] = np.maximum(night1_max, night2_max)

In [2956]:
asthma_children = extractChild(asthma_df)

for child in asthma_children:
    imputer = KNNImputer()
    child.loc[:,["HRMaxSleep"]] = np.round(imputer.fit_transform(child.loc[:,["HRMaxSleep"]]))
    child.loc[:,["HRMinSleep"]] = np.round(imputer.fit_transform(child.loc[:,["HRMinSleep"]]))
    child.loc[:,["HRAvgSleep"]] = np.round(imputer.fit_transform(child.loc[:,["HRAvgSleep"]]))

asthma_df = pd.concat(asthma_children)

/var/folders/cj/lqwb00w969q7gkbs7700mm300000gn/T/ipykernel_69173/4094238502.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  child.loc[:,["HRMaxSleep"]] = np.round(imputer.fit_transform(child.loc[:,["HRMaxSleep"]]))
/var/folders/cj/lqwb00w969q7gkbs7700mm300000gn/T/ipykernel_69173/4094238502.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  child.loc[:,["HRMinSleep"]] = np.round(imputer.fit_transform(child.loc[:,["HRMinSleep"]]))
/var/folders/cj/lqwb00w969q7gkbs7700mm300000gn/T/ipykernel_69173/409423850

## AVGHR_sleep (avg hr between 12am - 5am)

Do we need this?

## awakeDuration (during night), wakeUpCount, lightSleepDuration, deepSleepDuration
We should drop these


## Showing the % of the missing data

In [2957]:
percentNan = []
for col in asthma_df.columns:
    nan_percentage = asthma_df[col].isnull().sum() / len(asthma_df[col]) * 100
    percentNan.append((nan_percentage, col))

percentNan.sort(reverse=True)

percentNan

[(92.1796360820751, 'EventDay'),
 (76.38404955478126, 'EOS_weight_tot'),
 (76.38404955478126, 'EOS_treatmentburden_tot'),
 (76.38404955478126, 'EOS_school_tot'),
 (76.38404955478126, 'EOS_respiratory_tot'),
 (76.38404955478126, 'EOS_physical_tot'),
 (76.38404955478126, 'EOS_health_tot'),
 (76.38404955478126, 'EOS_emotional_tot'),
 (76.38404955478126, 'EOS_digestion_tot'),
 (76.38404955478126, 'EOS_bodyimage_tot'),
 (75.30003871467285, 'Eosinophils'),
 (73.94502516453736, 'Symptomscore'),
 (69.68641114982579, 'Serum_IgE'),
 (69.26054974835463, 'ACD6'),
 (67.36353077816493, 'Pseudomonas'),
 (67.36353077816493, 'BASELINE_weight_tot'),
 (67.36353077816493, 'BASELINE_treatmentburden_tot'),
 (67.36353077816493, 'BASELINE_school_tot'),
 (67.36353077816493, 'BASELINE_respiratory_tot'),
 (67.36353077816493, 'BASELINE_physical_tot'),
 (67.36353077816493, 'BASELINE_health_tot'),
 (67.36353077816493, 'BASELINE_emotional_tot'),
 (67.36353077816493, 'BASELINE_digestion_tot'),
 (67.36353077816493, 'B

## Drop columns

In [2958]:
asthma_df = asthma_df.drop(['EOS_weight_tot', 'EOS_treatmentburden_tot','EOS_school_tot','EOS_respiratory_tot','EOS_physical_tot','EOS_health_tot',
                            'EOS_emotional_tot','EOS_digestion_tot','EOS_bodyimage_tot','Eosinophils','Symptomscore','Serum_IgE','ACD6',
                            'BASELINE_weight_tot','BASELINE_treatmentburden_tot','BASELINE_school_tot','BASELINE_respiratory_tot',
                            'BASELINE_physical_tot','BASELINE_health_tot','BASELINE_emotional_tot','BASELINE_digestion_tot','BASELINE_bodyimage_tot',
                            'Pseudomonas','PancrInsuffi','sleep_score_child','screentime','school_yes_no','activity_score_parent',
                            'activity_score_child','ACD6score','ACD6_use','ACD5','ACD4','ACD3','ACD2','ACD1','EOS_PAQLQ_total',
                            'EOS_PAQLQ_symptoms','EOS_PAQLQ_emotion','EOS_PAQLQ_activity','EOS_ACQ6_score','BASELINE_PAQLQ_total',
                            'BASELINE_PAQLQ_symptoms','BASELINE_PAQLQ_emotion','BASELINE_PAQLQ_activity','BASELINE_ACQ6_score','AtopicAsthma','SABA',
                            'OtherClass','Oral_Steroid','Monoclonals','Leukotri_Mod','LABA','ICS','ExerciseSymp','AllergicRhin','grade_fvc',
                            'grade_fev1','Technique','pef_best','fvc_best','fvc1_percentage','fev1_ratio_percentage','fev1_ratio_best',
                            'fev1_percentage','fev1_best','EOS_PedsQL_score', 'wakeUpCount','lightSleepDuration','deepSleepDuration','awakeDuration',
                            'AVGHR_sleep','Ox','Nox','AVGHR_wake','BASELINE_PedsQL_score','HrAvgWake','Sports','totalSleepDuration','pollenTotal',
                            'NO2','NO','AgeDiagnosis','Family_his'], axis=1)

In [2959]:
asthma_df.to_csv(os.path.join('preprocessed_KNN_asthma.csv'), index=False)

In [2960]:
percentNan = []
for col in asthma_df.columns:
    nan_percentage = asthma_df[col].isnull().sum() / len(asthma_df[col]) * 100
    percentNan.append((nan_percentage, col))

percentNan.sort(reverse=True)

percentNan

[(92.1796360820751, 'EventDay'),
 (0.0, 'weekday'),
 (0.0, 'week'),
 (0.0, 'waketimeReport'),
 (0.0, 'urbanisation2'),
 (0.0, 'steps_hour_max'),
 (0.0, 'stepsTotalDetailed'),
 (0.0, 'stepsTotalDaily'),
 (0.0, 'steps23'),
 (0.0, 'steps22'),
 (0.0, 'steps21'),
 (0.0, 'steps20'),
 (0.0, 'steps19'),
 (0.0, 'steps18'),
 (0.0, 'steps17'),
 (0.0, 'steps16'),
 (0.0, 'steps15_19'),
 (0.0, 'steps15'),
 (0.0, 'steps14'),
 (0.0, 'steps13'),
 (0.0, 'steps12'),
 (0.0, 'steps11'),
 (0.0, 'steps10'),
 (0.0, 'steps09'),
 (0.0, 'steps08'),
 (0.0, 'steps07'),
 (0.0, 'steps06'),
 (0.0, 'steps05'),
 (0.0, 'steps04'),
 (0.0, 'steps03'),
 (0.0, 'steps02'),
 (0.0, 'steps01'),
 (0.0, 'steps00'),
 (0.0, 'school_year_edit'),
 (0.0, 'dayType'),
 (0.0, 'bedtimeReport'),
 (0.0, 'Weight'),
 (0.0, 'Vaccination'),
 (0.0, 'Urbanisation'),
 (0.0, 'Unnamed: 0'),
 (0.0, 'TX'),
 (0.0, 'TN'),
 (0.0, 'TG'),
 (0.0, 'SubjectNr'),
 (0.0, 'SmokeHome'),
 (0.0, 'SchoolYear'),
 (0.0, 'SQ'),
 (0.0, 'SP'),
 (0.0, 'RHX'),
 (0.0, 'RH')